In [5]:
import cv2
import os
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.callbacks import CSVLogger
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.optimizers import Adam
from models import *
from data_generator import *

train_frames_dir = './data/train_frames/'
train_masks_dir = './data/train_masks/'

val_frames_dir = './data/val_frames/'
val_masks_dir = './data/val_masks/'

INPUT_SHAPE = (256, 256, 3)
NO_CLASSES = 20

NO_OF_TRAINING_FRAMES = len(os.listdir(train_frames_dir))
NO_OF_VAL_FRAMES = len(os.listdir(val_frames_dir))

NO_OF_EPOCHS = 10

BATCH_SIZE = 4

In [6]:
model = segnet(num_classes=20, input_size=INPUT_SHAPE)

In [7]:
train_gen = data_gen(img_folder = train_frames_dir,
                    mask_folder = train_masks_dir,
                    batch_size = BATCH_SIZE,
                    num_classes = NO_CLASSES,
                    input_shape = INPUT_SHAPE)

val_gen = data_gen(img_folder = val_frames_dir,
                    mask_folder = val_masks_dir,
                    batch_size = BATCH_SIZE,
                    num_classes = NO_CLASSES,
                    input_shape = INPUT_SHAPE)

In [ ]:
results = model.fit_generator(train_gen, epochs=NO_OF_EPOCHS, 
                          steps_per_epoch = (NO_OF_TRAINING_FRAMES//BATCH_SIZE),
                          validation_data=val_gen, 
                          validation_steps=(NO_OF_VAL_FRAMES//BATCH_SIZE))

Epoch 1/10
743/743 [==============================] - 166s 224ms/step - loss: 1.0100 - acc: 0.7378 - val_loss: 0.8739 - val_acc: 0.7531
Epoch 2/10
743/743 [==============================] - 163s 219ms/step - loss: 0.7129 - acc: 0.7952 - val_loss: 0.7377 - val_acc: 0.7913
Epoch 3/10
743/743 [==============================] - 163s 220ms/step - loss: 0.6334 - acc: 0.8133 - val_loss: 0.6372 - val_acc: 0.8122
Epoch 4/10
496/743 [===================>..........] - ETA: 51s - loss: 0.5852 - acc: 0.8253